This notebook cleans .txt and .dta data for students performance on exams for BIO1111 and exports a csv which can be analyzed in an R script.
#### Author: Christopher Agard

In [1]:
import pandas as pd
import numpy as np
import os,glob
from soar import *

pd.options.display.max_columns = 50

# Setting source and output folders

In [2]:
def sourceFiles(filedict,source = 'worksource', output = 'workoutput'):
    filedict = filedict[input('Choose a filesystem: ')]
    filedict[source] = filedict[source].format(year=year,semester=semester,exNum=exNum)
    filedict[output] = filedict[output].format(year=year,semester=semester,exNum=exNum)
    return filedict

In [3]:
sources = {'tu':{'worksource': "C:/Users/craga/Google Drive/TU {year}/Exams/{semester}/Exam {exNum}/Input Data/",
    'workoutput': "C:/Users/tuh27554/Documents/Grading/{year}/{semester} EX{exNum}/Output Data/"},
        'gandolf':{'worksource': "C:/Users/craga/Google Drive/TU {year}/{semester}/Exams/Exam {exNum}/Input Data/",
    'workoutput': "C:/Users/craga/Google Drive/TU {year}/{semester}/Exams/Exam {exNum}/Output Data/"}}

In [4]:
year=input('which year? ')
semester=input('which semester? ')
exNum=input('which exam number? ')

which year? 2020
which semester? Spring
which exam number? 2


In [5]:
sources = sourceFiles(sources)

Choose a filesystem: gandolf


In [6]:
sources

{'worksource': 'C:/Users/craga/Google Drive/TU 2020/Spring/Exams/Exam 2/Input Data/',
 'workoutput': 'C:/Users/craga/Google Drive/TU 2020/Spring/Exams/Exam 2/Output Data/'}

# Getting and Cleaning Exam Data

In [7]:
os.chdir(sources['worksource'])
files = glob.glob('*.dta')
files

['marcexams_2019_2019-A2534.dta']

In [8]:
# os.chdir(source['worksource'])
df1=cleanSoarExam(files[0],
                  examNum=1,
                  soarSessions=[int(x) for x in input('Enter your SOAR sessions separated by commas: ').split(',')])
df1['version'] = 1
# df2=cleanSoarExam(files[1],examNum=1,soarSessions=[73,74,75])
# df2['version'] = 2
df = df1.loc[(df1.soar.isin([np.nan]+[int(x) for x in input('Enter your SOAR sessions separated by commas: ').split(',')]))&(df1.soarType.isin(['key','mine']))]#.append(df2)#remove students not in your sections
df.head()

Enter your SOAR sessions separated by commas: 73,74,75


C:\Users\craga\Documents\GitHub\Grading\soar.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  tmp['soar'] = soar


Enter your SOAR sessions separated by commas: 73,74,75


,tuid,last,first,middle,unnamed1,unnamed2,unnamed3,soar,ncorrect,examNumber,item_1,item_2,item_3,item_4,item_5,item_6,item_7,item_8,item_9,item_10,item_11,item_12,item_13,item_14,item_15,item_16,item_17,item_18,item_19,item_20,item_21,item_22,item_23,item_24,item_25,item_26,item_27,soarType,version
0,500 - BLU,EXAM - KEY,NNNNNN,N,NaN,NaN,NaN,NaN,27,1,3,2,4,4,4,1,3,1,3,2,2,2,1,4,2,2,3,4,2,2,3,1,1,2,1,2,4,key,1
9,915619341,RANJI,LAUREN,NaN,22.0,21.0,24.0,73.0,9,1,2,4,2,1,2,4,1,1,2,3,2,4,1,3,2,1,3,4,2,4,1,3,1,2,4,4,3,mine,1
10,915767393,QAZI,ZEMAR,NaN,22.0,20.0,9.0,75.0,10,1,3,4,3,4,4,4,2,4,3,3,3,4,1,4,1,2,4,4,1,4,3,2,1,3,3,1,2,mine,1
12,915637331,SATUNAS,KATELY,NaN,22.0,20.0,13.0,75.0,12,1,4,4,3,4,2,4,3,1,1,3,4,2,4,4,3,4,2,4,1,2,3,1,1,2,4,3,4,mine,1
14,914894157,NELMS,MATT,B,22.0,21.0,96.0,73.0,18,1,3,2,4,4,2,1,2,4,3,4,2,2,1,3,2,2,3,2,2,4,2,1,1,2,1,2,3,mine,1


In [9]:
df.soar.unique()

array([nan, 73., 75., 74.])

Now we change the dir to the outputfolder and save the cleaned, merged file.

In [10]:
itemcols = df.columns[df.columns.str.contains('item')]
itemcols

Index(['item_1', 'item_2', 'item_3', 'item_4', 'item_5', 'item_6', 'item_7',
       'item_8', 'item_9', 'item_10', 'item_11', 'item_12', 'item_13',
       'item_14', 'item_15', 'item_16', 'item_17', 'item_18', 'item_19',
       'item_20', 'item_21', 'item_22', 'item_23', 'item_24', 'item_25',
       'item_26', 'item_27'],
      dtype='object')

In [11]:
for col in itemcols:
    df[col] = df[col].apply(letters2num)

C:\Users\craga\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [18]:
os.chdir(sources['workoutput'])
df.to_csv(input('Enter filename prefix: ')+'.csv')

Enter filename prefix: Spring 2020 Exam 2 Data for my sections


# Analyzing Exam Data

1. [Most Frequently Wrong](#Most-Frequently-Wrong-Answers)

## Strange values for items

In reviewing the data I noted some values outside of the expected 1-4.  Could you help me understand what these are?

In [13]:
all_responses = pd.concat([pd.Series(df[col].unique()) for col in itemcols]).unique()
strange_response = [resp for resp in all_responses if resp not in ['A','B','C','D','E','1','2','3','4','5']]
strange_resp_item=[col for col in itemcols if df.loc[df[col].isin(strange_response),col].shape[0]>0]
print("The strange responses {} were found for the following items:\n{}".format(strange_response, strange_resp_item))


The strange responses [] were found for the following items:
[]


## Most Frequently Wrong Answers

Here we identify the columns containing information about responses.

In [14]:
df.soarType.value_counts()

mine    76
key      5
Name: soarType, dtype: int64

Now we apply this function to create a ranked list of items by order of the number of my students getting them wrong.

In [15]:
wrongcount_v1_mine = pd.DataFrame()
for soar in [x for x in df.soar.unique() if not np.isnan(x)]:
    for col in itemcols:
        wrongcount_v1_mine = wrongcount_v1_mine.append(countwrong(df= df.loc[df.soar==soar],
                                                                  col = col, 
                                                                  ver = 1,
                                                                  compind=['mine','other']))
    wrongcount_v1_mine = wrongcount_v1_mine.sort_values(['number','item'],
                                                        ascending=False)\
    .reset_index(drop = True)
    # wrongcount_v1_mine.index = wrongcount_v1_mine.item
    # wrongcount_v1_mine= wrongcount_v1_mine.drop(columns = 'item')
    print('wrongcount_{}:\n{}'.format(int(soar),wrongcount_v1_mine))
    total_wrongtmp = wrongcount_v1_mine.set_index(['item']).sum(axis=1)
    total_wrongtmp.to_csv('total wrong {}.csv'.format(int(soar)),index = True)
    print('total_wrong_{}:\n{}'.format(int(soar),
                                       total_wrongtmp.sort_values(ascending=False)))

wrongcount_73:
       item  number
0    item_2      33
1    item_8      29
2    item_5      24
3   item_19      24
4    item_3      23
5   item_25      22
6   item_15      22
7   item_26      21
8   item_14      21
9   item_24      20
10  item_16      20
11  item_20      19
12   item_4      18
13  item_11      17
14   item_7      16
15  item_10      16
16  item_22      15
17   item_9      14
18   item_6      13
19  item_17      13
20  item_13      13
21  item_27      12
22  item_21      12
23  item_23      10
24  item_12      10
25   item_1      10
26  item_18       9
total_wrong_73:
item
item_2     33
item_8     29
item_5     24
item_19    24
item_3     23
item_25    22
item_15    22
item_26    21
item_14    21
item_24    20
item_16    20
item_20    19
item_4     18
item_11    17
item_7     16
item_10    16
item_22    15
item_9     14
item_6     13
item_17    13
item_13    13
item_27    12
item_21    12
item_1     10
item_23    10
item_12    10
item_18     9
dtype: int64


C:\Users\craga\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  from ipykernel import kernelapp as app


wrongcount_75:
       item  number
0    item_2      33
1    item_8      29
2    item_5      24
3   item_19      24
4    item_3      23
5   item_25      22
6   item_15      22
7   item_26      21
8   item_14      21
9   item_24      20
10  item_16      20
11  item_20      19
12   item_4      18
13   item_2      17
14  item_11      17
15   item_7      16
16  item_10      16
17  item_22      15
18   item_9      14
19   item_8      14
20   item_5      14
21  item_19      14
22   item_6      13
23   item_3      13
24  item_17      13
25  item_13      13
26  item_27      12
27  item_26      12
28  item_25      12
29  item_24      12
30  item_21      12
31  item_11      11
32  item_23      10
33  item_16      10
34  item_15      10
35  item_13      10
36  item_12      10
37   item_1      10
38  item_18       9
39   item_6       8
40  item_20       7
41  item_10       7
42   item_7       6
43  item_27       6
44  item_22       6
45  item_17       6
46  item_14       6
47  item_21       5
48  i

## Printing the Keys

In [16]:
# import numpy as np
v1keys = df.loc[(df.soar.isnull())&(df.soarType=='key')&(df.version==1),itemcols].iloc[0]
# v2keys = df.loc[(df.soarType=='key')&(df.version==2),itemcols].iloc[0]
keys = pd.DataFrame(data = {'version1':v1keys})#,'version2':v2keys})
keys.to_csv(input('Enter filename prefix for the keys: ')+'.csv',index = False)
keys

Enter filename prefix for the keys: Exam 2020 Spring Exam 2 keys


,version1
item_1,B
item_2,C
item_3,A
item_4,A
item_5,A
item_6,D
item_7,B
item_8,D
item_9,B
item_10,C


# Evaluating Student Performance

## Score distribution
Now we look at scores by section. 

In [17]:
descCols= ['mean','min','25%','50%','75%','max']
dfDesc = df.loc[(df.soarType!='key')].groupby(['soarType','soar']).ncorrect.describe()
dfDesc = pd.concat([dfDesc[['count']],dfDesc[descCols].apply(lambda x: x/25*100)],axis=1)
dfDesc

count       mean   min   25%   50%   75%   max
soarType soar                                                
mine     73.0   35.0  53.600000  36.0  40.0  52.0  64.0  84.0
         74.0   23.0  54.260870  28.0  48.0  60.0  66.0  72.0
         75.0   18.0  56.888889  32.0  48.0  56.0  64.0  88.0

## Item Analysis
1. [Percent choice for each item](#Percent-choice-for-each-item)

os.chdir(source['worksource'])
df20 = pd.read_csv('Spring 2020v1.csv')
dfcomp = pd.read_csv('Cross-Walk 2019 Fall to 2018 Fall Exams.csv')
dfcomp

### Percent choice for each item

df_item1 = pd.DataFrame()
for col in itemcols:
    df_item1 = df_item1.append(df.loc[df.version==1,col].value_counts(normalize=True))
df_item1 = df_item1.applymap(lambda x: "{:.0f}".format(x*100))
df_item1 = df_item1.reset_index().rename(columns={'index':'F19E1'})
df_item1['F19E1'] = df_item1.F19E1.apply(lambda x: int(x[5:]))
df_item1 = df_item1.sort_values('F19E1')
df_item1['F18E1'] = dfcomp.F18E1
df_item1.to_csv('Fall 2019v1.csv')
df_item1

df_item = pd.DataFrame()
for col in itemcols:
    df_item = df_item.append(df.loc[df.version==2,col].value_counts(normalize=True))
df_item = df_item.applymap(lambda x: "{:.0f}".format(x*100))
df_item

df18

dfcomp